In [1]:
import pandas as pd
import openpyxl
import sqlite3

In [2]:
firefox_conn = sqlite3.connect("places.sqlite")

In [3]:
firefox_df = pd.read_sql_query('''
SELECT h.place_id, p.url, count(h.place_id)
FROM moz_historyvisits h
JOIN moz_places p ON h.place_id = p.id
WHERE p.url like '%duckduckgo%'
OR p.url like '%google%search%'
GROUP BY h.place_id;
''', firefox_conn)

In [4]:
firefox_counts = firefox_df.url.str.split(pat=r'[\/\+&=%\.]',expand=True).stack().value_counts()
firefox_counts.head()

              9554
com           8476
https:        7996
duckduckgo    7736
?q            7726
dtype: int64

In [5]:
firefox_df2 = firefox_counts.rename_axis('keywords').reset_index(name='count')
firefox_df2.head()

,unique_values,counts
0,,9554
1,com,8476
2,https:,7996
3,duckduckgo,7736
4,?q,7726


In [6]:
firefox_df2.to_excel('firefox_history.xlsx')

In [7]:
google_conn = sqlite3.connect("History")
google_df = pd.read_sql_query('''
SELECT id, url, title, visit_count, typed_count, last_visit_time, hidden
FROM urls
WHERE url like '%google%search%'
OR url like '%duckduckgo%q%'
ORDER by visit_count DESC;
''', google_conn)

In [8]:
google_counts = google_df.url.str.split(pat=r'[\/\+&=%\.]',expand=True).stack().value_counts()
google_counts.head()

          9054
0         1572
google    1328
https:    1310
chrome    1287
dtype: int64

In [9]:
google_df = google_counts.rename_axis('keywords').reset_index(name='count')
google_df.head()

,unique_values,counts
0,,9054
1,0,1572
2,google,1328
3,https:,1310
4,chrome,1287


In [10]:
google_df.to_excel('google_history.xlsx')